In [ ]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D

In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define the Residual Block
def residual_block(x, filters, stride=1):
    shortcut = x

    # First convolutional layer
    x = layers.Conv2D(filters, (3, 3), strides=stride, padding='same')(x)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)

    # Second convolutional layer
    x = layers.Conv2D(filters, (3, 3), padding='same')(x)
    x = layers.BatchNormalization()(x)

    # Shortcut connection
    if stride != 1 or shortcut.shape[-1] != filters:
        shortcut = layers.Conv2D(filters, (1, 1), strides=stride, padding='valid')(shortcut)
        shortcut = layers.BatchNormalization()(shortcut)

    x = layers.Add()([x, shortcut])
    x = layers.ReLU()(x)
    return x

# Define the ResNet-50-like model
def ResNet50(input_shape, num_classes):
    input_tensor = layers.Input(shape=input_shape)

    # Initial Convolutional Layer
    x = layers.Conv2D(64, (7, 7), strides=(2, 2), padding='same')(input_tensor)
    x = layers.BatchNormalization()(x)
    x = layers.ReLU()(x)
    x = layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same')(x)

    # Stack of Residual Blocks
    block_sizes = [3, 4, 6, 3]  # Number of residual blocks in each stage
    filters = [64, 128, 256, 512]

    for stage, num_blocks in enumerate(block_sizes):
        for _ in range(num_blocks):
            if stage == 0 and _ == 0:
                x = residual_block(x, filters[stage], stride=1)
            else:
                x = residual_block(x, filters[stage], stride=2 if _ == 0 else 1)

    # Global Average Pooling
    x = layers.GlobalAveragePooling2D()(x)

    # Fully Connected Layers
    x = layers.Dense(num_classes, activation='softmax')(x)

    model = Model(inputs=input_tensor, outputs=x)
    return model

# Example usage
input_shape = (224, 224, 3)  # Example input shape for RGB images
num_classes = 1000  # Number of output classes
model = ResNet50(input_shape, num_classes)

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_2 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d_36 (Conv2D)          (None, 112, 112, 64)         9472      ['input_2[0][0]']             
                                                                                                  
 batch_normalization_36 (Ba  (None, 112, 112, 64)         256       ['conv2d_36[0][0]']           
 tchNormalization)                                                                                
                                                                                                  
 re_lu_33 (ReLU)             (None, 112, 112, 64)         0         ['batch_normalization_36

In [ ]:
# Data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Load your dataset using the ImageDataGenerator
train_generator = datagen.flow_from_directory(
    '/content/drive/MyDrive/Colab Notebooks/Leaf Disease/Tomato_Train',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

validation_generator = datagen.flow_from_directory(
    '//content/drive/MyDrive/Colab Notebooks/Leaf Disease/Tomato_test',
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

Found 8950 images belonging to 10 classes.
Found 2295 images belonging to 10 classes.


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=len(train_generator),
    epochs=40,
    validation_data=validation_generator,
    validation_steps=len(validation_generator)
)